In [ ]:

#import all required library
import os
import cv2
import numpy as np
import random as rn
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt


In [ ]:
# import all tensorflow and keras library 

import tensorflow as tf
import keras
from keras import initializers
from keras import regularizers
from keras import constraints
from keras import backend as K
from keras.activations import elu
from keras.optimizers import Adam
from keras.models import Sequential
from keras.engine import Layer, InputSpec
from keras.utils.generic_utils import get_custom_objects
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from keras.layers import Dense, Conv2D, Flatten, GlobalAveragePooling2D, Dropout,MaxPooling2D,BatchNormalization,GlobalMaxPooling2D

In [ ]:
# set seed for reproducability
seed=1234
rn.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
os.environ["PYTHONHASHSEED"]=str(seed)

In [ ]:
base_model=tf.keras.applications.MobileNetV2(input_shape=(256,256,3),include_top=False,weights="imagenet")

In [ ]:
# Set all layer trainable
for layer in base_model.layers:
    layer.trainable=True

In [ ]:
# Build Model according to your output
def build_model():
    model=Sequential()
    model.add(base_model) 
    model.add(GlobalAveragePooling2D())
    model.add(Dropout(0.3))  # change dropout get better result
    model.add(Dense(1,activation="sigmoid")) # output is 0 or 1 binary 
    
    # now compile model
    # we are using adam you can use other to optimize better
    optimizer=keras.optimizers.Adam(learning_rate=0.00005,beta_1=0.9,beta_2=0.999,amsgrad=False)
    # This is important part of model 
    # Number of positive image is less than number of negative image
    # we use AUC metrics
    metrics=tf.keras.metrics.AUC(name="auc")
    model.compile(loss="binary_crossentropy",optimizer=optimizer,metrics=metrics)
    print(model.summary())
    return model

model=build_model()
    
    

In [ ]:
df=pd.read_csv("../input/jpeg-melanoma-256x256/train.csv")

In [ ]:
df # we will use only image_name and target

In [ ]:

a,b=np.unique(df["target"],return_counts=True)

In [ ]:
a

In [ ]:
b # 32542 negative image and only 584 positive image


In [ ]:
# split train dataframe into training and validation

In [ ]:
from sklearn.model_selection import train_test_split
train,valid=train_test_split(df,test_size=0.2)

In [ ]:

train,test=train_test_split(train,test_size=0.01)

In [ ]:
train["image_name_jpg"]=train["image_name"]+".jpg"
test["image_name_jpg"]=test["image_name"]+".jpg"
valid["image_name_jpg"]=valid["image_name"]+".jpg"

In [ ]:
# now convert target in string
train["target"]=train["target"].astype(str)
test["target"]=test["target"].astype(str)
valid["target"]=valid["target"].astype(str)

In [ ]:
# now create train_generator and validation_generator
# you can add other augmentation example vertical_flip, random cropping ,etc to get bettter accuracy
train_datagen=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255,
    horizontal_flip=True
    )

In [ ]:
test_datagen=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255) # we have to divide by 255 for testing in android app

In [ ]:
train_generator =train_datagen.flow_from_dataframe(
    dataframe=train,
    directory="../input/jpeg-melanoma-256x256/train/",
    x_col="image_name_jpg", # name +".jpg"
    y_col="target",
    target_size=(256,256),
    batch_size=32,
    class_mode="binary"
)

In [ ]:
validation_generator =test_datagen.flow_from_dataframe(
    dataframe=valid,
    directory="../input/jpeg-melanoma-256x256/train/",
    x_col="image_name_jpg",
    y_col="target",
    target_size=(256,256),
    batch_size=16,
    class_mode="binary"
)

In [ ]:
#train model
model.fit_generator(
    train_generator,
    epochs=3, #you have set or use early stopping to stop before overfiting
    shuffle=True,
    validation_data=validation_generator
    # can also define callback to save model when gets best accuracy
)

In [ ]:
# This is most basic version of training with decent accuracy 

In [ ]:
# After 3 epoch it start to overfit 
#When the training is done save model in tflite formate which is much faster but accuracy decreases
converter=tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model=converter.convert()
# save model
with open ("model.tflite","wb") as f:
    f.write(tflite_model)
